<a href="https://colab.research.google.com/github/leticiabalbi/AI/blob/main/MichalskiTrain/LTN_MichalskiTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
Zip_ref = zipfile.ZipFile('/content/logictensornetworks.zip', 'r')
Zip_ref.extractall('/content/')
Zip_ref.close()

In [ ]:
!pip install -e '/content/logictensornetworks'
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logictensornetworks as ltn
!pip install pylint
import pylint
import logging; logging.basicConfig(level=logging.INFO)

In [59]:
tf.config.run_functions_eagerly(True)

# Import data

In [43]:
df = pd.read_csv('/content/data.csv')

#Fill nan values
df = df.fillna(-2).astype('float32')
df['direcao'] = df['direcao'].astype('int32')

df.head(10)

,quant_vagoes,quant_cargas,v1_rodas,v1_comprimento,v1_formato,v1_quantidade_cargas,v1_formato_carga,v2_rodas,v2_comprimento,v2_formato,v2_quantidade_cargas,v2_formato_carga,v3_rodas,v3_comprimento,v3_formato,v3_quantidade_cargas,v3_formato_carga,v4_rodas,v4_comprimento,v4_formato,v4_quantidade_cargas,v4_formato_carga,v5_rodas,v5_comprimento,v5_formato,v5_quantidade_cargas,v5_formato_carga,ret_ret,ret_tri,ret_hex,ret_cir,tri_tri,tri_hex,tri_cir,hex_hex,hex_cir,cir_cir,direcao
0,5.0,4.0,2.0,1.0,5.0,0.0,-2.0,2.0,1.0,2.0,3.0,3.0,2.0,0.0,9.0,1.0,4.0,3.0,1.0,2.0,1.0,2.0,2.0,0.0,2.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,1
1,4.0,3.0,2.0,1.0,5.0,0.0,-2.0,2.0,0.0,2.0,1.0,4.0,2.0,0.0,8.0,1.0,3.0,2.0,0.0,1.0,2.0,1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
2,4.0,2.0,2.0,1.0,5.0,0.0,-2.0,2.0,0.0,2.0,1.0,1.0,2.0,0.0,6.0,1.0,4.0,3.0,1.0,1.0,1.0,4.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1
3,5.0,3.0,2.0,1.0,5.0,0.0,-2.0,2.0,0.0,8.0,1.0,4.0,2.0,0.0,3.0,1.0,4.0,2.0,0.0,4.0,1.0,2.0,2.0,0.0,2.0,1.0,3.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1
4,4.0,3.0,2.0,1.0,5.0,0.0,-2.0,2.0,0.0,3.0,1.0,4.0,3.0,1.0,1.0,1.0,4.0,2.0,0.0,1.0,1.0,1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
5,3.0,2.0,2.0,1.0,5.0,0.0,-2.0,2.0,1.0,1.0,3.0,1.0,2.0,0.0,2.0,1.0,4.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,0
6,4.0,2.0,2.0,1.0,5.0,0.0,-2.0,2.0,0.0,3.0,1.0,1.0,2.0,0.0,2.0,1.0,4.0,2.0,1.0,7.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,0
7,3.0,2.0,2.0,1.0,5.0,0.0,-2.0,3.0,1.0,1.0,1.0,3.0,2.0,0.0,2.0,1.0,1.0,-2.0,-2.0,-2.0,-2.0,3.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
8,5.0,2.0,2.0,1.0,5.0,0.0,-2.0,2.0,0.0,8.0,1.0,1.0,3.0,1.0,7.0,1.0,3.0,2.0,0.0,2.0,1.0,-2.0,2.0,0.0,8.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
9,3.0,1.0,2.0,1.0,5.0,0.0,-2.0,2.0,0.0,2.0,1.0,3.0,2.0,1.0,2.0,2.0,3.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0


# Train/test split

In [44]:
df = df.sample(frac=1).reset_index(drop=True)
df_test = df.loc[:1]
df_train = df.loc[2:]

In [46]:
labels_train = df_train.pop("direcao")
labels_test = df_test.pop("direcao")
batch_size = 8
ds_train = tf.data.Dataset.from_tensor_slices((df_train,labels_train)).batch(batch_size)
ds_test = tf.data.Dataset.from_tensor_slices((df_test,labels_test)).batch(batch_size)

# LTN

In [77]:
class MLP(tf.keras.Model):
    """Model that returns logits."""
    def __init__(self, n_classes, hidden_layer_sizes=(16,16,8)):
        super(MLP, self).__init__()
        self.denses = [tf.keras.layers.Dense(s, activation="elu") for s in hidden_layer_sizes]
        self.dense_class = tf.keras.layers.Dense(n_classes)
        self.dropout = tf.keras.layers.Dropout(0.2)
        
    def call(self, inputs, training=False):
        x = inputs
        for dense in self.denses:
            x = dense(x)
            x = self.dropout(x, training=training)
        return self.dense_class(x)

logits_model = MLP(37)
p = ltn.Predicate(ltn.utils.LogitsToPredicateModel(logits_model,single_label=True))

In [78]:
class_West = ltn.constant(0)
class_East = ltn.constant(1)

In [79]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")

In [80]:
formula_aggregator = ltn.fuzzy_ops.Aggreg_pMeanError(p=2)

@tf.function
def axioms(features, labels, training=False):
    x_West = ltn.variable("x_West",features[labels==0])
    x_East = ltn.variable("x_East",features[labels==1])
    axioms = [
        Forall(x_West,p([x_West,class_West],training=training)),
        Forall(x_East,p([x_East,class_East],training=training))
    ]
    axioms = tf.stack(axioms)
    sat_level = formula_aggregator(axioms)
    return sat_level, axioms

In [81]:
for features, labels in ds_test:
    print("Initial sat level %.5f"%axioms(features,labels)[0])
    break

Initial sat level 0.02101


# Training

In [82]:
metrics_dict = {
    'train_sat_kb': tf.keras.metrics.Mean(name='train_sat_kb'),
    'test_sat_kb': tf.keras.metrics.Mean(name='test_sat_kb'),
    'train_accuracy': tf.keras.metrics.CategoricalAccuracy(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
}

In [83]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
@tf.function
def train_step(features, labels):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(features, labels, training=True)[0]
        loss = 1.-sat
    gradients = tape.gradient(loss, p.trainable_variables)
    optimizer.apply_gradients(zip(gradients, p.trainable_variables))
    sat = axioms(features, labels)[0] # compute sat without dropout
    metrics_dict['train_sat_kb'](sat)
    # accuracy
    predictions = logits_model(features)
    metrics_dict['train_accuracy'](tf.one_hot(labels,2),predictions)
    
@tf.function
def test_step(features, labels):
    # sat
    sat = axioms(features, labels)[0]
    metrics_dict['test_sat_kb'](sat)
    # accuracy
    predictions = logits_model(features)
    metrics_dict['test_accuracy'](tf.one_hot(labels,2),predictions)

In [84]:
from collections import defaultdict

def train(
        epochs,
        metrics_dict, 
        ds_train, 
        ds_test, 
        train_step, 
        test_step,
        track_metrics=1,
        csv_path=None,
        scheduled_parameters=defaultdict(lambda : {})
    ):
    """
    Args:
        epochs: int, number of training epochs.
        metrics_dict: dict, {"metrics_label": tf.keras.metrics instance}.
        ds_train: iterable dataset, e.g. using tf.data.Dataset.
        ds_test: iterable dataset, e.g. using tf.data.Dataset.
        train_step: callable function. the arguments passed to the function
            are the itered elements of ds_train.
        test_step: callable function. the arguments passed to the function
            are the itered elements of ds_test.
        csv_path: (optional) path to create a csv file, to save the metrics.
        scheduled_parameters: (optional) a dictionary that returns kwargs for
            the train_step and test_step functions, for each epoch.
            Call using scheduled_parameters[epoch].
    """
    template = "Epoch {}"
    for metrics_label in metrics_dict.keys():
        template += ", %s: {:.4f}" % metrics_label
    if csv_path is not None:
        csv_file = open(csv_path,"w+")
        headers = ",".join(["Epoch"]+list(metrics_dict.keys()))
        csv_template = ",".join(["{}" for _ in range(len(metrics_dict)+1)])
        csv_file.write(headers+"\n")
    
    for epoch in range(epochs):
        for metrics in metrics_dict.values():
            metrics.reset_states()

        for batch_elements in ds_train:
            train_step(*batch_elements,**scheduled_parameters[epoch])
        for batch_elements in ds_test:
            test_step(*batch_elements,**scheduled_parameters[epoch])

        metrics_results = [metrics.result() for metrics in metrics_dict.values()]
        if epoch%track_metrics == 0:
            print(template.format(epoch,*metrics_results))
        if csv_path is not None:
            csv_file.write(csv_template.format(epoch,*metrics_results)+"\n")
            csv_file.flush()
    if csv_path is not None:
        csv_file.close()

In [85]:
EPOCHS = 300

train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    train_step,
    test_step,
    csv_path="train_results.csv",
    track_metrics=20
)



Epoch 0, train_sat_kb: 0.0160, test_sat_kb: 0.0221, train_accuracy: 0.0000, test_accuracy: 0.0000
Epoch 20, train_sat_kb: 0.0336, test_sat_kb: 0.0404, train_accuracy: 0.0000, test_accuracy: 0.5000
Epoch 40, train_sat_kb: 0.0672, test_sat_kb: 0.0637, train_accuracy: 0.8750, test_accuracy: 1.0000
Epoch 60, train_sat_kb: 0.1601, test_sat_kb: 0.1400, train_accuracy: 1.0000, test_accuracy: 0.5000
Epoch 80, train_sat_kb: 0.4253, test_sat_kb: 0.3709, train_accuracy: 1.0000, test_accuracy: 0.5000
Epoch 100, train_sat_kb: 0.7434, test_sat_kb: 0.6312, train_accuracy: 1.0000, test_accuracy: 1.0000
Epoch 120, train_sat_kb: 0.8781, test_sat_kb: 0.7447, train_accuracy: 1.0000, test_accuracy: 1.0000
Epoch 140, train_sat_kb: 0.9518, test_sat_kb: 0.6100, train_accuracy: 1.0000, test_accuracy: 0.5000
Epoch 160, train_sat_kb: 0.9781, test_sat_kb: 0.5978, train_accuracy: 1.0000, test_accuracy: 0.5000
Epoch 180, train_sat_kb: 0.9897, test_sat_kb: 0.6086, train_accuracy: 1.0000, test_accuracy: 0.5000
Epoch 